# Light up the world, precious nerds!

Why stumble around in the dark when you need to visit the bathroom at night?
Make the Pi turn on lights upon detecting motion in the dark.
The GPIO pins hook up with the Pi with sensors to give eyes and ears.
We shall see how as we work through the schematics.


## What is GPIO?

A Maker project is a system with components that exchange messages with the Pi. Sensors convey messages about external conditions like temperature, light and pressure. An actuators, like a motor, receives a message to energize. The GPIO pins provide the interface for messaging exchanges. 

In the old days, a taxi-meter had a flag to indicate whether a taxi was vacant or not. Once the taxi had a fare, the operator manually rotated the flag to indicate the taxi was not available any more. A GPIO pin has a similar function as the taxi-meter flag. It conveys a simple binary message - high or low. ![Flag](images/taxi_meter_flag.jpg)

When a sensor conveys information to the Pi, it signals a "High" or "Low" on a GPIO pin. Thus a temperature sensor may indicate high or low temperature. Or a moisture sensor may indicate dry or wet soil. To recieve information, a GPIO pin must be in listening mode, i.e. "Input".

The Pi can energize an actuator via GPIO pin conveying a "High" or "Low". For this, the GPIO pin must be in transmitting mode, i.e. "Output". A GPIO pin may directly power a low-power device, like an LED light. More often, the device's power requirements dictate that it possess own power supply. In that case, the GPIO signal the device to energize using own power source rather than supply the power for consumption. In this way, a Pi can control appliances around the home like a gas furnace or a washing machine, which require mains power.

## GPIO Output: Blink an LED

A message relayed via GPIO takes the form of an electrical voltage. When the voltage is raised to 3.3 V, the pin signals a "High" as if flag were raised. When the voltage is lowered to 0 V, the pin signals a "Low" as if flag were lowered. The 3.3 V on the pin is sufficient to power a small LED light. Wire up the circuit like so. ![BlinkLED](images/ozWiz_dancingLights_e_blinkLED.png)

The schematic shows a Raspberry Pi with a cartoon of the GPIO pins as they appear on the board. The pins are numbered 1 through 40 in the order of appearance. Some pins simple serve to provide an output voltage (3.3 V or 5 V) and a reference 0 V (Ground). A few GPIO pins haver additional specialized uses in communicating complex messages. Pins also have a GPIO ID number that the Maker can use when refering to pin numbers in code, being careful to specify upfront which numbering scheme is in use.